In [1]:
import os
import torch
from random import choice
from diffusers import (
    StableDiffusionPipeline,
    EulerAncestralDiscreteScheduler,
    PNDMScheduler, DPMSolverMultistepScheduler)
from tqdm import tqdm

from uuid import uuid4
from diffusers.utils import load_image, make_image_grid

2024-09-13 20:56:11.618412: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-13 20:56:13.279428: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/opt/conda/lib/python3.11/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/opt/conda/lib/python3.11/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a f

In [2]:
def generate_images(diffusion_model, syntetic_base_path, steps_number_all=100, resolution=512):
    for target, prompt in [
        (1, 'a photo of large malignant mole. Malignant'),
        (1, 'a photo of large malignant moles. Malignant'),
        (1, 'a photo of multiple malignant moles. Malignant'),
        (1, 'a photo of multiple lesion moles. Malignant'),
        (1, 'a photo of one bening moles and one malignant mole. Malignant')
    ]:
        for sampler in [PNDMScheduler, DPMSolverMultistepScheduler]:
            diffusion_model.scheduler = sampler.from_config(diffusion_model.scheduler.config)
    
            for i in range(steps_number_all):
                image_tmp = diffusion_model(
                    prompt=prompt,
                    height=resolution,
                    width=resolution,
                    num_inference_steps=choice([i for i in range(20,40)]),
                    guidance_scale = choice([3,4,5,6,7,8]),
                    negative_prompt="",
                    # generator = [torch.Generator("cuda").manual_seed(1)]
                )[0][0]
                
                overall_index = str(uuid4())
                image_tmp.save(os.path.join(syntetic_base_path, f"{overall_index}___{target}.png"))

In [3]:
base_folder_to_save = "../data/artifacts/syntetic_base_folds_final"
os.makedirs(base_folder_to_save, exist_ok=True)

finetunes_path = "..." #path to your finetunes_dir
finetunes_epochs_to_use = ['epoch-000035', 'epoch-000040', 'epoch-000045', 'epoch-000050']

In [ ]:
images_list = []
for finetune_name in os.listdir(finetunes_path):
    tmp = [i for i in finetune_name.split('_') if '' != i]
    seed_tmp = tmp[-3] #-2
    fold_tmp = tmp[-1]
    print(seed_tmp, fold_tmp)

    base_folder_to_save_tmp_hr = os.path.join(base_folder_to_save, seed_tmp, fold_tmp, "hr")
    os.makedirs(base_folder_to_save_tmp_hr, exist_ok=True)

    base_folder_to_save_tmp_lr = os.path.join(base_folder_to_save, seed_tmp, fold_tmp, "lr")
    os.makedirs(base_folder_to_save_tmp_lr, exist_ok=True)
    for epoch_model in finetunes_epochs_to_use:
        diffusion_model = StableDiffusionPipeline.from_pretrained( #from_single_file
            # "Derm-T2IM.safetensors",
            os.path.join(finetunes_path, finetune_name, epoch_model),
            torch_dtype=torch.float16,
            safety_checker=None,
            use_safetensors=True
        )
        diffusion_model = diffusion_model.to("cuda:0")
        
        generate_images(diffusion_model, base_folder_to_save_tmp_hr, steps_number_all=100, resolution=512)
        
        generate_images(diffusion_model, base_folder_to_save_tmp_lr, steps_number_all=100, resolution=128)
        
        diffusion_model.to('cpu')
        